In [1]:
# ATSIT 2019: dimensional speech emotion recognition from text feature

# uncomment these to run on CPU only
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras.backend as K
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, LSTM, TimeDistributed, Bidirectional, Embedding,GRU, Dropout, Flatten, concatenate, Convolution1D, BatchNormalization
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from keras.layers.convolutional import SeparableConv2D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
import codecs
from keras.optimizers import SGD, Adam, RMSprop
import io
import random as rn
import tensorflow as tf
from keras.layers.core import Dense, Activation
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
    
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape
# remove outlier
# outlier = np.array([1674, 3427, 5086, 5093, 5096, 5104, 7821])
# mask = np.ones(len(vad), np.bool)
# mask[outlier] = 0
# vad = vad[mask]
# x_train_mocap = x_train_mocap[mask]


Using TensorFlow backend.
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/anaconda3/envs/ser36/lib/python3.6/sit

(10039, 200, 189, 1)

In [5]:
# API model, if use RNN, first two rnn layer must return_sequences=True
def api_model(a,b,c):
    
    input_mocap = Input(shape=(200,189,1))
    model_mocap1 = SeparableConv2D(32, 3, strides=(2, 2), padding="same")(input_mocap)
    model_mocap2 = Dropout(0.2)(model_mocap1)
    model_mocap3 = Activation('relu')(model_mocap2)
    model_mocap4 = SeparableConv2D(64, 3,strides=(2, 2), padding="same")(model_mocap3)
    model_mocap5 = Dropout(0.2)(model_mocap4)
    model_mocap6 = Activation('relu')(model_mocap5)
    model_mocap7 = SeparableConv2D(128, 3,strides=(2, 2), padding="same")(model_mocap6)
    model_mocap8 = Dropout(0.2)(model_mocap7) 
    model_mocap9 = Activation('relu')(model_mocap8)
    model_mocap10 = SeparableConv2D(64, 3,strides=(2, 2), padding="same")(model_mocap9)   
    model_mocap11 = Dropout(0.2)(model_mocap10)  
    model_mocap12 = Activation('relu')(model_mocap11)
    model_mocap13 = SeparableConv2D(32, 3,strides=(2, 2), padding="same")(model_mocap12)  
    model_mocap14 = Dropout(0.2)(model_mocap13) 
    model_mocap15 = Activation('relu')(model_mocap14)
    model_mocap = Flatten()(model_mocap15) 
    #model_mocap = Dropout(dropout_rate)(model_mocap16)

    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_mocap) for name in target_names]

    model = Model(input_mocap, model_combined) 

    model.compile(loss=ccc_loss, 
                  loss_weights={'v': a, 'a': b, 'd': c},
                  optimizer='rmsprop', metrics=[ccc])
    return model

#def main(alpha, beta, gamma):
#model = api_model(1, 1, 1)
#odel = api_model(0.1, 0.5, 0.4)
#model.summary()
# main function for for LOSO


In [6]:
def main():
    model = api_model(1,1,1)
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model.fit( x_train_mocap[:7869], 
                      vad[:7869].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model.evaluate( x_train_mocap[7869:], vad[7869:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.1, 0.5, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main()
        weight.append([time, best[-3:], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('mocap_CNNTranspose.csv', header=['time', 'best', 'ave'])


Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 64s 10ms/step - loss: 2.8466 - v_loss: 0.9654 - a_loss: 0.9390 - d_loss: 0.9415 - v_ccc: 0.0343 - a_ccc: 0.0609 - d_ccc: 0.0581 - val_loss: 2.6964 - val_v_loss: 0.9385 - val_a_loss: 0.8776 - val_d_loss: 0.8842 - val_v_ccc: 0.0580 - val_a_ccc: 0.1285 - val_d_ccc: 0.1171
Epoch 2/50
6295/6295 [==============================] - 63s 10ms/step - loss: 2.3949 - v_loss: 0.7466 - a_loss: 0.8183 - d_loss: 0.8290 - v_ccc: 0.2531 - a_ccc: 0.1811 - d_ccc: 0.1708 - val_loss: 2.6381 - val_v_loss: 0.8935 - val_a_loss: 0.8792 - val_d_loss: 0.8643 - val_v_ccc: 0.0978 - val_a_ccc: 0.1272 - val_d_ccc: 0.1369
Epoch 3/50
6295/6295 [==============================] - 60s 10ms/step - loss: 2.1135 - v_loss: 0.5745 - a_loss: 0.7528 - d_loss: 0.7859 - v_ccc: 0.4252 - a_ccc: 0.2471 - d_ccc: 0.2143 - val_loss: 2.6201 - val_v_loss: 0.9152 - val_a_loss: 0.8214 - val_d_loss: 0.8817 - val_v_ccc: 0.0783 - val_a_ccc: 0

6295/6295 [==============================] - 61s 10ms/step - loss: 1.7987 - v_loss: 0.4272 - a_loss: 0.6817 - d_loss: 0.6897 - v_ccc: 0.5729 - a_ccc: 0.3181 - d_ccc: 0.3103 - val_loss: 2.4983 - val_v_loss: 0.8091 - val_a_loss: 0.8298 - val_d_loss: 0.8625 - val_v_ccc: 0.1872 - val_a_ccc: 0.1757 - val_d_ccc: 0.1387
Epoch 10/50
6295/6295 [==============================] - 63s 10ms/step - loss: 1.7719 - v_loss: 0.4164 - a_loss: 0.6767 - d_loss: 0.6783 - v_ccc: 0.5833 - a_ccc: 0.3231 - d_ccc: 0.3217 - val_loss: 2.4416 - val_v_loss: 0.8006 - val_a_loss: 0.7962 - val_d_loss: 0.8439 - val_v_ccc: 0.1922 - val_a_ccc: 0.2099 - val_d_ccc: 0.1562
Epoch 11/50
6295/6295 [==============================] - 66s 11ms/step - loss: 1.7367 - v_loss: 0.4158 - a_loss: 0.6631 - d_loss: 0.6581 - v_ccc: 0.5845 - a_ccc: 0.3370 - d_ccc: 0.3418 - val_loss: 2.5902 - val_v_loss: 0.8060 - val_a_loss: 0.8550 - val_d_loss: 0.9277 - val_v_ccc: 0.1839 - val_a_ccc: 0.1538 - val_d_ccc: 0.0721
Epoch 12/50
6295/6295 [========

6295/6295 [==============================] - 66s 11ms/step - loss: 1.5245 - v_loss: 0.3639 - a_loss: 0.5856 - d_loss: 0.5745 - v_ccc: 0.6362 - a_ccc: 0.4141 - d_ccc: 0.4252 - val_loss: 2.5419 - val_v_loss: 0.8183 - val_a_loss: 0.8569 - val_d_loss: 0.8663 - val_v_ccc: 0.1764 - val_a_ccc: 0.1462 - val_d_ccc: 0.1355
Epoch 35/50
6295/6295 [==============================] - 60s 9ms/step - loss: 1.5377 - v_loss: 0.3639 - a_loss: 0.5921 - d_loss: 0.5821 - v_ccc: 0.6360 - a_ccc: 0.4080 - d_ccc: 0.4184 - val_loss: 2.4994 - val_v_loss: 0.8044 - val_a_loss: 0.8502 - val_d_loss: 0.8475 - val_v_ccc: 0.1922 - val_a_ccc: 0.1539 - val_d_ccc: 0.1544
Epoch 36/50
6295/6295 [==============================] - 61s 10ms/step - loss: 1.5248 - v_loss: 0.3622 - a_loss: 0.5852 - d_loss: 0.5784 - v_ccc: 0.6380 - a_ccc: 0.4153 - d_ccc: 0.4219 - val_loss: 2.4715 - val_v_loss: 0.8247 - val_a_loss: 0.8125 - val_d_loss: 0.8345 - val_v_ccc: 0.1682 - val_a_ccc: 0.1930 - val_d_ccc: 0.1673
Epoch 37/50
6295/6295 [=========

Epoch 19/50
6295/6295 [==============================] - 60s 10ms/step - loss: 1.7241 - v_loss: 0.4010 - a_loss: 0.6506 - d_loss: 0.6718 - v_ccc: 0.5988 - a_ccc: 0.3494 - d_ccc: 0.3277 - val_loss: 2.4764 - val_v_loss: 0.8596 - val_a_loss: 0.8081 - val_d_loss: 0.8094 - val_v_ccc: 0.1355 - val_a_ccc: 0.1960 - val_d_ccc: 0.1921
Epoch 20/50
6295/6295 [==============================] - 58s 9ms/step - loss: 1.7119 - v_loss: 0.4009 - a_loss: 0.6509 - d_loss: 0.6599 - v_ccc: 0.5990 - a_ccc: 0.3489 - d_ccc: 0.3401 - val_loss: 2.4930 - val_v_loss: 0.8451 - val_a_loss: 0.7790 - val_d_loss: 0.8681 - val_v_ccc: 0.1491 - val_a_ccc: 0.2258 - val_d_ccc: 0.1321
Epoch 21/50
6295/6295 [==============================] - 59s 9ms/step - loss: 1.6992 - v_loss: 0.3858 - a_loss: 0.6565 - d_loss: 0.6570 - v_ccc: 0.6142 - a_ccc: 0.3437 - d_ccc: 0.3429 - val_loss: 2.4383 - val_v_loss: 0.8616 - val_a_loss: 0.7457 - val_d_loss: 0.8358 - val_v_ccc: 0.1357 - val_a_ccc: 0.2604 - val_d_ccc: 0.1656
Epoch 22/50
6295/6295

6295/6295 [==============================] - 60s 10ms/step - loss: 1.5724 - v_loss: 0.3657 - a_loss: 0.6052 - d_loss: 0.6014 - v_ccc: 0.6343 - a_ccc: 0.3947 - d_ccc: 0.3987 - val_loss: 2.5396 - val_v_loss: 0.8653 - val_a_loss: 0.8200 - val_d_loss: 0.8546 - val_v_ccc: 0.1301 - val_a_ccc: 0.1844 - val_d_ccc: 0.1459
Epoch 45/50
6295/6295 [==============================] - 62s 10ms/step - loss: 1.5601 - v_loss: 0.3687 - a_loss: 0.5979 - d_loss: 0.5932 - v_ccc: 0.6311 - a_ccc: 0.4021 - d_ccc: 0.4067 - val_loss: 2.5127 - val_v_loss: 0.8583 - val_a_loss: 0.8056 - val_d_loss: 0.8488 - val_v_ccc: 0.1366 - val_a_ccc: 0.1987 - val_d_ccc: 0.1520
Epoch 46/50
2170/2170 [==============================] - 4s 2ms/step
[0.2072499543428421, 0.2128121256828308, 0.17282257974147797]
2
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 56s 9ms/step - loss: 2.8978 - v_loss: 0.9794 - a_loss: 0.9668 - d_loss: 0.9505 - v_ccc: 0.0203 - a_ccc: 0.0329 - d_ccc: 0

...

6295/6295 [==============================] - 65s 10ms/step - loss: 2.9097 - v_loss: 0.9818 - a_loss: 0.9711 - d_loss: 0.9579 - v_ccc: 0.0185 - a_ccc: 0.0293 - d_ccc: 0.0425 - val_loss: 2.8840 - val_v_loss: 0.9953 - val_a_loss: 0.9103 - val_d_loss: 0.9804 - val_v_ccc: 0.0031 - val_a_ccc: 0.0931 - val_d_ccc: 0.0198
Epoch 2/50
6295/6295 [==============================] - 64s 10ms/step - loss: 2.6968 - v_loss: 0.8769 - a_loss: 0.9218 - d_loss: 0.8988 - v_ccc: 0.1230 - a_ccc: 0.0785 - d_ccc: 0.1018 - val_loss: 2.7857 - val_v_loss: 0.9318 - val_a_loss: 0.9067 - val_d_loss: 0.9375 - val_v_ccc: 0.0555 - val_a_ccc: 0.0967 - val_d_ccc: 0.0621
Epoch 3/50
6295/6295 [==============================] - 64s 10ms/step - loss: 2.2776 - v_loss: 0.5832 - a_loss: 0.8673 - d_loss: 0.8268 - v_ccc: 0.4167 - a_ccc: 0.1324 - d_ccc: 0.1732 - val_loss: 2.7322 - val_v_loss: 0.9028 - val_a_loss: 0.8935 - val_d_loss: 0.9274 - val_v_ccc: 0.0850 - val_a_ccc: 0.1101 - val_d_ccc: 0.0727
Epoch 4/50
6295/6295 [===========

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6295/6295 [==============================] - 66s 10ms/step - loss: 1.7236 - v_loss: 0.4020 - a_loss: 0.6619 - d_loss: 0.6602 - v_ccc: 0.5982 - a_ccc: 0.3383 - d_ccc: 0.3399 - val_loss: 2.5601 - val_v_loss: 0.8688 - val_a_loss: 0.8669 - val_d_loss: 0.8190 - val_v_ccc: 0.1212 - val_a_ccc: 0.1359 - val_d_ccc: 0.1828
Epoch 16/50
6295/6295 [==============================] - 59s 9ms/step - loss: 1.6918 - v_loss: 0.3992 - a_loss: 0.6433 - d_loss: 0.6490 - v_ccc: 0.6006 - a_ccc: 0.3567 - d_ccc: 0.3508 - val_loss: 2.5405 - val_v_loss: 0.8698 - val_a_loss: 0.7729 - val_d_loss: 0.9009 - val_v_ccc: 0.1238 - val_a_ccc: 0.2354 - val_d_ccc: 0.1004
Epoch 17/50
6295/6295 [==============================] - 62s 10ms/step - loss: 1.6863 - v_loss: 0.3941 - a_loss: 0.6496 - d_loss: 0.6428 - v_ccc: 0.6058 - a_ccc: 0.3508 - d_ccc: 0.3571 - val_loss: 2.4585 - val_v_loss: 0.8529 - val_a_loss: 0.8012 - val_d_loss: 0.8005 - val_v_ccc: 0.1359 - val_a_ccc: 0.2036 - val_d_ccc: 0.2019
Epoch 18/50
6295/6295 [=========

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2170/2170 [==============================] - 3s 1ms/step
[0.24438178539276123, 0.2468314915895462, 0.24509485065937042]
19
